In [42]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_curve
import numpy as np
import joblib
import os

# Define dataset path
dataset_dir = '/content/drive/MyDrive/new parkinson'  # Update with actual path

# Image dimensions & batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Data Augmentation & Preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split dataset into 80% train, 20% validation
)

# Load Training & Validation Data
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

# Load MobileNetV2 (pretrained)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

# Custom Model Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = Dropout(0.5)(x)
output_layer = Dense(1, activation='sigmoid')(x)  # <-- FIX: Connect to `x`

# Build Model
model = Model(inputs=base_model.input, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train Model
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

class_weights = {0: 1.0, 1: np.sum(train_generator.classes == 0) / np.sum(train_generator.classes == 1)}

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    class_weight=class_weights,
    callbacks=[early_stopping]
)

# Fine-Tuning: Unfreeze some layers of the base model
base_model.trainable = True
for layer in base_model.layers[:100]:  # Keep first 100 layers frozen
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    class_weight=class_weights,
    callbacks=[early_stopping]
)

# Save the model
model.save('parkinsons_mobilenetv2_finetuned.h5')

# Load Validation Data for Testing
test_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

# Predictions
probs = model.predict(test_generator)
y_test = test_generator.classes

# Find Optimal Threshold
fpr, tpr, thresholds = roc_curve(y_test, probs)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

# Save threshold
joblib.dump(optimal_threshold, 'optimal_threshold.pkl')

print(f"Optimal Threshold: {optimal_threshold:.2f}")

Found 825 images belonging to 2 classes.
Found 205 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 230s 9s/step - accuracy: 0.5334 - loss: 3.0045 - val_accuracy: 0.5756 - val_loss: 2.9059
Epoch 2/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 75s 3s/step - accuracy: 0.7029 - loss: 2.6963 - val_accuracy: 0.5512 - val_loss: 2.7917
Epoch 3/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.7849 - loss: 2.5056 - val_accuracy: 0.5707 - val_loss: 2.7069
Epoch 4/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 76s 3s/step - accuracy: 0.8450 - loss: 2.3185 - val_accuracy: 0.5854 - val_loss: 2.6210
Epoch 5/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 66s 3s/step - accuracy: 0.8718 - loss: 2.1894 - val_accuracy: 0.6244 - val_loss: 2.5332
Epoch 6/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 66s 3s/step - accuracy: 0.8880 - loss: 2.0519 - val_accuracy: 0.6293 - val_loss: 2.4603
Epoch 7/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 66s 3s/step - accuracy: 0.8959 - loss: 1.9606 - val_accuracy: 0.6098 - val_loss: 2.3648
Epoch 8/20
26/26 ━━━━━━━━━━━━━━━━━━━━ 76s 3s/step - accuracy: 0.9121 - loss: 1.8457 - val_accuracy: 0.6341 - val_loss

Found 205 images belonging to 2 classes.
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step
Optimal Threshold: 0.75


In [56]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
print(os.path.exists('parkinsons_mobilenetv2_finetuned.h5'))  # Should be True
print(os.path.exists('optimal_threshold.pkl'))  # Should be True



True
True


In [60]:
def predict_parkinson(image_path):
    model = load_model('parkinsons_mobilenetv2_finetuned.h5')
    optimal_threshold = joblib.load('optimal_threshold.pkl')

    if not os.path.exists(image_path):
        print(f" Error: Image '{image_path}' not found!")
        return

    print(f"Loading image: {image_path}")
    img = load_img(image_path, target_size=(224, 224))  # Ensure correct format
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    print(" Making prediction...")
    prob = model.predict(img_array)[0][0]
    prediction = " Parkinson's Detected" if prob >= optimal_threshold else " No Parkinson's"

    print(f"\n Prediction: {prediction}")
    print(f" Confidence Score: {prob:.2f}")

    return prediction, prob

# Test the function
predict_parkinson('/content/FSPGR_BRAVO_212.png')
predict_parkinson('/content/O_Ax_DWI_1000b_009.png')


Loading image: /content/FSPGR_BRAVO_212.png
 Making prediction...
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

 Prediction:  Parkinson's Detected
 Confidence Score: 0.90


Loading image: /content/O_Ax_DWI_1000b_009.png
 Making prediction...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step

 Prediction:  No Parkinson's
 Confidence Score: 0.00


(" No Parkinson's", 0.001561562)

In [61]:
from google.colab import drive
drive.mount('/content/drive')

# Define the save path in Google Drive
drive_save_path = '/content/drive/MyDrive/parkinsons_mobilenetv2_finetuned.h5'

# Save the model to Google Drive
model.save(drive_save_path)
print(f"✅ Model saved to: {drive_save_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model saved to: /content/drive/MyDrive/parkinsons_mobilenetv2_finetuned.h5
